# IMU_timestamp_check

This notebook checks the consistency of the timestamps injected on the trigger pin.

The trigger pin frequency is at 100 Hz from a decimated 20 ppm clock. We should be able to see this clock by checking the timestamps

In [ ]:
import sys
from mcap.reader import make_reader
from mcap_ros2.decoder import DecoderFactory
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Bag path
BAG="/data/high_altitude_test/vnav/rosbag2_2024_08_05-22_11_09/rosbag2_2024_08_05-22_11_09_0.mcap"

# timesyncin_arr = np.zeros(
with open(BAG, "rb") as f:
    reader = make_reader(f, decoder_factories=[DecoderFactory()])

    # print(reader.get_summary().statistics)
    num_msgs = reader.get_summary().statistics.channel_message_counts[1]
    arr_syncincnt = np.zeros(num_msgs)
    arr_timesyncin = np.zeros(num_msgs) #ns
    arr_timestartup = np.zeros(num_msgs)
    arr_timegps = np.zeros(num_msgs)
    arr_timegpspps = np.zeros(num_msgs)

    for i, (schema, channel, message, ros_msg) in enumerate(reader.iter_decoded_messages()):
        if i == 0:
            print(f"{channel.topic} {schema.name} [{message.log_time}]: {ros_msg}")
        msg=ros_msg
        arr_syncincnt[i] = msg.syncincnt
        arr_timesyncin[i] = msg.timesyncin
        arr_timestartup[i] = msg.timestartup
        arr_timegps[i] = msg.timegps
        arr_timegpspps[i] = msg.timegpspps
        if (i % 1000 == 0):
            print(f"{i} - {i/num_msgs}")
    arr_timesyncin /= 1e3 #us

In [ ]:
# General plot of the syncincounter
plt.figure()
plt.plot(arr_timesyncin/5e3)
plt.plot(arr_syncincnt)
plt.show()

In [ ]:
# Let's split the three moments where we send triggers, and the last continuous trigger
fig, ax = plt.subplots(nrows=3, ncols=1)

for i in range(3):
    int_s = np.where(arr_syncincnt==(1+300*i))[0][0]
    int_e = np.where(arr_syncincnt==(300*(i+1)))[0][3]
    print(f"{i} - s: {int_s} - e: {int_e} - diff: {int_e - int_s +1}")
    first_int = arr_timesyncin[int_s:int_e+1]
    first_d = first_int[::4]
    x_ax = np.arange(0, 300, 1)
    ax[i].plot(x_ax, first_d)
    m, b = np.polyfit(x_ax, first_d, 1)
    print(f"first int: {first_d[0]} - last int: {first_d[-1]}, drift: {m*300/3e6*1e6} ppm")
plt.show()
    # print((first_d))

#plt.figure()
#plt.plot()
# plt.plot(arr_syncincnt[idx])
#plt.show()

print(len(arr_timesyncin[t0_s:t0_e]))

# splits = timesyncin_p > 10
#print(timesyncin_p[101]/1e9)

print(idx1)


In [ ]:
(1199 - 1166)/3e6 * 1e6
